# Question types



In [ ]:
import pathlib
import warnings

import geoviews.tile_sources as gvts
import hvplot.pandas
import hvplot.xarray
import panel as pn
import pooch
import requests

from coastal_dynamics.geometries import geo_bbox
from coastal_dynamics.questions.factory import QuestionWidgetFactory
from coastal_dynamics.questions.multiple_choice import MultipleChoiceQuestion
from coastal_dynamics.questions.multiple_selection import MultipleSelectionQuestion
from coastal_dynamics.questions.numeric import NumericQuestion
from coastal_dynamics.questions.text import TextQuestion
from coastal_dynamics.visualization import DynamicWavePlot

pn.extension()

## Examples for all four question types

#### Multiple Choice

In [ ]:
mcq = MultipleChoiceQuestion(
    question_name="Q1: Simple Multiple Choice Question",
    question_text="What coastal landform do you typically find in tide-dominated environments?",
    question_options={
        "a": "Mudflats",
        "b": "Open coasts",
        "c": "Cliffed coasts",
        "d": "Mixed sand and gravel beaches",
    },
    question_answer="a",
).serve()

#### Multiple Selection

In [ ]:
msq = MultipleSelectionQuestion(
    question_name="Q1: Simple Multiple Selection Question",
    question_text="Select all morphological systems that are commonly in coastal areas.",
    question_options={
        "a": "Beaches",
        "b": "Glaciers",
        "c": "Estuaries",
        "d": "Mountains",
        "e": "Sand", 
        "f": "Mud", 
    },
    question_answers=["a", "c"],
).serve()

#### Numeric question

In [ ]:
nq = NumericQuestion(
    question_name="Q3: Simple Numeric Question",
    question_text="What is the PSU of the global ocean?",
    question_answer=35,
    **{"precision": 0},
).serve()


#### Text Question

In [ ]:
TextQuestion(
    question_name="Q4: Some simple text question",
    question_text="How do you call the force that generates the tide?",
    question_answer="Gravitational pull",
).serve()

### Some basis visualization blocks

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning, module="holoviews.core.data")
wave_plot = DynamicWavePlot(amplitude_range=(1, 3, 0.5), wavelength_range=(5, 20, 0.1))
wave_plot

In [ ]:
base_plot = gvts.EsriImagery(width=900, height=500)
base_plot

### Combine using panel

In [ ]:
pn.Column(wave_plot, base_plot)

## Make questions from a database (json) that is formatted like below

In [ ]:
import json

import pooch

template_questions_href = pooch.retrieve(
    "https://coclico.blob.core.windows.net/coastal-dynamics/questions/template.json",
    known_hash="9ef56da50895d243cc5bdb2184f838a33706195522fbb8992767defd6b8d0b96",
)

with pathlib.Path(template_questions_href).open("r") as f:
    questions = json.loads(f.read())

In [ ]:
questions["q1"]

### This is a factory class that builds the quetions based on the question type

In [ ]:
question_data = questions["q1"]
factory = QuestionWidgetFactory(question_data)
q1 = factory.create_question_widget().serve()

### Combine all in a dashboard - works also for other questions

In [ ]:
all_questions_panel = pn.Column(sizing_mode="stretch_width")

# Add each question's panel to the main Panel column
for key, question_data in questions.items():
    question_panel = QuestionWidgetFactory(question_data).create_question_widget().serve()
    all_questions_panel.append(question_panel)

# Create a wave plot and a base plot
wave_plot = DynamicWavePlot(
    amplitude_range=(1, 3, 0.5), wavelength_range=(5, 20, 0.1)
)
base_plot = gvts.EsriImagery(width=900, height=500)

# Combine all components into a single dashboard
dashboard = pn.Column(
    pn.pane.Markdown("# Questions"),
    all_questions_panel,
    pn.pane.Markdown("# Coastal Dynamics Interactive Dashboard"),
    base_plot,
    wave_plot.__panel__(),  # Assuming DynamicWavePlot has a __panel__ method
    sizing_mode="stretch_width",
)
# or dashboard.show() for the browser
dashboard